In [6]:
import pandas as pd
from model import Brain
from submodels import factories
import matplotlib.pyplot as plt
from jf.db import DB
from lib.plot import plot_2d_positions_colors
from lib.sde.grn import GRNMain

In [8]:
grn = GRNMain(5, 0, 5)
ccls = factories["grn1"](grn=grn)

bb = Brain(time_step=0.5, verbose=True, start_population=4,
        cell_cls=ccls.generate, end_time=60, start_time=50, silent=False)
bb.run()

Ticking abs : 50.0, step : 0.5
Ticking abs : 50.5, step : 0.5
Ticking abs : 51.0, step : 0.5
Duplicating cell 0 located in 0
Duplicating cell 1 located in 1
Duplicating cell 2 located in 2
Duplicating cell 3 located in 3
Duplicating cell 4 located in 4
Duplicating cell 5 located in 5
Duplicating cell 6 located in 6
Duplicating cell 7 located in 7
Duplicating cell 8 located in 8
Duplicating cell 9 located in 9
Duplicating cell 10 located in 10
Duplicating cell 11 located in 11
Duplicating cell 12 located in 12
Duplicating cell 13 located in 13
Duplicating cell 14 located in 14
Duplicating cell 15 located in 15
Ticking abs : 51.5, step : 0.5
Duplicating cell 16 located in 17
Removing 17 16
Removing 18 19
Removing 19 18
Removing 20 21
Removing 21 20
Removing 22 23
Duplicating cell 23 located in 22
Duplicating cell 24 located in 25
Removing 25 24
Removing 26 27
Removing 27 26
Removing 28 29
Duplicating cell 29 located in 28
Duplicating cell 30 located in 31
Removing 31 30
Removing 32 33
Du

In [4]:
ref = pd.read_csv("output/results/setup_basic/export/ref_basic.csv")  # ref is a mean

In [5]:
ref

,Unnamed: 0,time,size_type_Cycling,size_type_PostMitotic
0,0,49.0,36.0,0.0
1,1,49.5,41.6,5.2
2,2,50.0,46.4,7.2
3,3,50.5,51.8,5.0
4,4,51.0,56.2,4.0
...,...,...,...,...
77,77,87.5,173.6,46.6
78,78,88.0,165.8,52.4
79,79,88.5,160.4,47.2
80,80,89.0,157.2,41.2


In [9]:
bb.stats

,progenitor_pop_size,size_type_Cell,time,whole_pop_size
50.0,16.0,16.0,50.0,16.0
50.5,16.0,16.0,50.5,16.0
51.0,32.0,32.0,51.0,32.0
51.5,24.0,24.0,51.5,44.0
52.0,10.0,10.0,52.0,49.0


In [17]:
def shrink_and_align_stats(stats, ref, max_step=None):
    min_step = max(min(stats.time), min(ref.time))
    if max_step is None:
        max_step = max(ref.time)
    
    new_stats = stats[(stats.time >= min_step) & (stats.time <= max_step)].set_index("time")
    new_ref = ref[(ref.time >= min_step) & (ref.time <= max_step)].set_index("time")
    
    return new_stats, new_ref

In [20]:
# compare only the progenitor_pop_size
def evaluating_function(stats, ref, max_step=None):
    """
    The evaluation function for progenitor population size.
    The lower is the better
    :param stats: the stats of the bb after running
    """
    colname_ref = "size_type_Cycling"
    colname_stats = "progenitor_pop_size"
    # scale
    stats, ref = shrink_and_align_stats(stats, ref, max_step=max_step)
    
    min_step = min(ref.index)  # ref.index is the time
    stats[colname_stats] = stats[colname_stats] / stats[colname_stats].get(min_step)
    ref[colname_ref] = ref[colname_ref] / ref[colname_ref].get(min_step)
    
    err = 0
    for t in ref.index:
        val_ref = ref[colname_ref].get(t, 0)
        val_stats = stats[colname_stats].get(t, 0)
        err += (val_ref - val_stats)**2 / val_ref
        
    return err

In [21]:
evaluating_function(bb.stats, ref)

377.0420929046707